In [ ]:
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ipywidgets as widgets
import matplotlib.pyplot as plt

import functions as helpers

pio.templates.default = "seaborn"
pd.options.plotting.backend = "plotly"

%matplotlib inline

In [ ]:
%store -r df

In [ ]:
age_ranges = ['18-24', '25-34', '35-44', '45-54', '55-64', '65-74','75-84', '85-94']

figs = []
m = 25
for age_range in age_ranges:
    fig = df[df['ageRange'] == age_range].groupby(['hour','series']).median().unstack('series')['steps'].plot.line(markers=False, line_shape='spline', width=300, height=250, title=age_range)
    fig.update_layout(annotations=[], overwrite=True)
    fig.update_layout(showlegend=False, margin=dict(t=m,l=m,b=m,r=m))
    fig.update_layout(xaxis = dict(
        tickvals = [0,6,12,18,23],
        ticktext = ['00:00', '06:00', '12:00', '18:00', '23:00']
    ), yaxis_title=None, xaxis_title=None)
    figs.append(go.FigureWidget(fig))
        
widgets.VBox([widgets.HBox([figs[0], figs[1], figs[2]]),widgets.HBox([figs[3], figs[4], figs[5]]), widgets.HBox([figs[6], figs[7]])])


In [ ]:
non_workers = df[df['work_age'] == False]
plot = (non_workers.groupby(['hour','series'])
   .median().unstack('series')['steps'].plot.line(title='Non workers',line_shape='spline', height=200))
plot.update_layout(xaxis = dict(
        tickmode = 'array',
        tickvals = [0,24, 48, 72, 96, 120, 144, 168],
        ticktext = ['Mon', 'Tue', 'Wed', 'Thur', 'Fri', 'Sat', 'Sun']
    ))

In [ ]:
workers = df[df['work_age'] == True]
steps = workers.groupby(['hour','series']).median().unstack('series')['steps']
plot = steps.plot.line(title='Workers', line_shape='spline', height=900, markers=True)
plot.update_layout(xaxis = dict(
        tickmode = 'array',
        tickvals = [0,6,12,18,23],
        ticktext = ['00:00', '06:00', '12:00', '18:00', '23:00']
    ))

periods = [{'from':5,'to':9,'label':'06:00 - 08:59'},{'from':9,'to':14,'label':'09:00 - 13:59'},{'from':14,'to':23,'label':'14:00 - 23:59'}]
for p in periods:
    middle = p['from'] + ((p['to'] - p['from']) / 2)
    stepsBefore = steps['before'][p['from']:p['to'] + 1].sum()
    stepsAfter = steps['after'][p['from']:p['to'] + 1].sum()
    annotate_diff(plot, middle, 580, p['label'], stepsBefore, stepsAfter)
    
    plot.add_vrect(x0=p['from'] + 0.05, x1=p['to'] - 0.05, opacity=0.15, layer="below", line_width=2, line_dash="dash")
    
plot.show()